In [4]:
import pulp
from pulp import*
import pandas as pd



#### formulation4 exact 3 team all other constraint

In [6]:
def optimization_form_4 (k,I:list,cov_ub:float,Y:dict,cons_stack_num ):
    '''
    I: we solve for ith entry sequentially; for i in I
    cov_ub: uperbound of pairwise cov
    '''
    prob = pulp.LpProblem(("opt"), pulp.LpMaximize)
    x = pulp.LpVariable.dicts("X",((i, j) for i in range(1,k) for j in all_player_list), 0 , 1,cat='Integer')
    # t[i,l] indicator: for a lineup i, if there are any player comes from team l
    ## this is for constraint |team| >=2
    t = pulp.LpVariable.dicts("T",((i, l) for i in range(1,k) for l in team_list), 0 , 1,cat='Integer')

    # v[i,l] complete line indicator (a line has 3player from same line)
    # w for partial line (2 player)
    v = pulp.LpVariable.dicts("V",((i, team) for i in range(1,k) for team in team_list), 0 , 1,cat='Integer')
    w = pulp.LpVariable.dicts("W",((i, team) for i in range(1,k) for team in team_list), 0 , 1,cat='Integer')
    # order_stack
    o = pulp.LpVariable.dicts("O",((i, team,stack) for i in range(1,k) for team in team_list for stack in range(8)), 0 , 1,cat='Integer')
    
    #obj
    prob += lpSum(x[(I[0],j)]* mu[j] for j in all_player_list) 
    #constraints
    # sum(salary)<50000 for each lineup
    for i in I:
        prob += lpSum(x[(i,j)] * salary_dict[j] for j in all_player_list) <=50000
    # each lineup has 10 player
    for i in I:
        prob += lpSum(x[(i,j)] for j in all_player_list) == 10
    # for a lineup  2 pitchers
    for i in I:
        prob += lpSum(x[(i,j)] for j in p_list) == 2
    # 1c, 1 oneb 1 twob 1 threeb 1ss 3 of
    for i in I:
        prob += lpSum(x[(i,j)] for j in c_list) == 1
        prob += lpSum(x[(i,j)] for j in oneb_list) == 1
        prob += lpSum(x[(i,j)] for j in twob_list) == 1
        prob += lpSum(x[(i,j)] for j in threeb_list) == 1
        prob += lpSum(x[(i,j)] for j in ss_list) == 1
        prob += lpSum(x[(i,j)] for j in of_list) == 3
    #4.2 team constrant : 
    ## players for each lineup must come from at least 2 team
    ## each team cannot exceed 5 player
    
    for i in I:
        for l in team_list:
            prob += 5*t[(i,l)] >= lpSum(x[(i,j)] for j in team_dict[l])
            prob += t[(i,l)] <= lpSum(x[(i,j)] for j in team_dict[l])
    for i in I:       
        prob += lpSum(t[(i,l)] for l in team_list) >= 3
        prob += lpSum(t[(i,l)] for l in team_list) <= 5                  

    #pitchers stacking
    ## force the goalie variable xik to be zero if the lineup has any skater opposing goalie k
    for i in I:    
        for j in p_list:
            prob += lpSum(x[(i,l)] for l in hitters_team_dict[p_opponent_dict[j]]) <= 5*(1-x[i,j])  


    #line stacking
    ## force each lineup has one complete line :5 hitters from a team, two partial line

#    for i in I:
#        for team in team_list: 
#            prob += 5 * v[(i,team)] <= lpSum(x[(i,j)] for j in hitters_team_dict[team]) 
#            prob += v[(i,l)] <= lpSum(x[(i,j)] for j in hitters_team_dict[l])
#        prob += lpSum(v[(i,team)] for team in team_list) == 1

    for i in I:
        for team in team_list: 
            prob += 2 * w[(i,team)] <= lpSum(x[(i,j)] for j in hitters_team_dict[team]) 
            prob += w[(i,l)] <= lpSum(x[(i,j)] for j in hitters_team_dict[l])
        prob += lpSum(w[(i,team)] for team in team_list) >= 2       
    #consecutive stacking
    for i in I:
        for team in team_list:
            for stack in range(8):
                prob += cons_stack_num * o[(i,team,stack)] <= lpSum(x[(i,j)] for j in order_stack_dict[team,stack]) #4 consq ok
                prob += o[(i,team,stack)] <= lpSum(x[(i,j)] for j in order_stack_dict[team,stack])
        prob += lpSum(o[(i,team,stack)] for team in team_list) == 1

        
    
    #4.4 overlap
    # from previously calculated entry we could get a table Y[i,j], 
    # relax overlap constraint for first entry
    if I[0] == 1:
        prob.solve()
        
        status = prob.status
        if status !=1:
            print(I)
            print(status)
        varsdict = {}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                entry = int(key.split(",_")[0][-1])
                player = str(key.split(",_")[1][1:-2])
                Y[(entry,player)] = value
        return(status,varsdict,Y)
    if I[0]>=2:
        for i in range(1,I[0]):
            prob += lpSum(Y[(i,j)]*x[(I[0],j)] for j in all_player_list) <= cov_ub
            #cov_ub+=0.1
        prob.solve(GLPK())
        status = prob.status
        if prob.status != 1:
            print(I)
            print(status)
        varsdict ={}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                if I[0]>=100:
                    entry = int(key.split(",_")[0][-1])+10*int(key.split(",_")[0][-2])+100*int(key.split(",_")[0][-3])
                elif I[0]>=10:
                    entry = int(key.split(",_")[0][-1])+10*int(key.split(",_")[0][-2])
                else:
                    entry = int(key.split(",_")[0][-1])
                player = key.split(",_")[1][1:-2]
                Y[(entry,player)] = value
        return(status,varsdict,Y)            

#### formulation_5 3team no goalie stacking

### def sequential_optimization_structure

In [7]:
def sequential_opt(k, cov_ub, optimization_form, cons_stack_num):
    '''
    k_entry: number of entry we want
    result: a dataframe representing result
    cov_ub: a list for pairwise covariance 
    optimization_form: which formulation of optimization
    '''

    progress = 0
    YY={}
    for i in range(1,k):
        for j in all_player_list:
            YY[(i,j)] = 0
    for i in [[x] for x in range(1,k)]:
        progress+=1
        print(str(progress)+"-th entry started>>>")
        (status,varsdict,YY) = optimization_form (k+1,i,cov_ub,YY, cons_stack_num)
    print(sum(YY.values()))    
    result_table = {}
    for i in range(1,k):
        result_table[i] = []
    for x,y in YY.items():
        if int(y)==1:
            result_table[x[0]].append(x[1])
    result = pd.DataFrame.from_dict(result_table)
    result.columns = ["entry" +str(i) for i in range(1,k)]
    result
    
    return result.T

## use data from DailyFantasyNerd, bend projection with ROTOGRINDER

In [8]:
hitters_data = pd.read_csv("DFN MLB Hitters DK 4_8.csv")
pitchers_data = pd.read_csv("DFN MLB Pitchers DK 4_8.csv")
hitters_data = hitters_data[hitters_data["Proj FP"]>0]
print(hitters_data.shape)
print(pitchers_data.shape)

(135, 31)
(16, 30)


In [9]:
hitters_data = hitters_data.rename(index=str, columns={"Player Name": "Last Name",
                                                       "Pos": "Position",
                                                     "Opp": "Opponent",
                                                     "PP": "Power_Play",
                                                     "Proj FP": "Projection"})
pitchers_data = pitchers_data.rename(index=str, columns={"Player Name": "Last Name", 
                                                     "Pos": "Position",
                                                     "Opp": "Opponent",
                                                     "PP": "Power_Play",
                                                     "Proj FP": "Projection"})  

hitters_data['Opponent'] = hitters_data['Opponent'].str.replace('@', '', regex=True)
pitchers_data['Opponent'] = pitchers_data['Opponent'].str.replace('@', '', regex=True)

hitters_data["Last Name"] = hitters_data["Last Name"].replace(' ', '_', regex=True)
pitchers_data["Last Name"] = pitchers_data["Last Name"].replace(' ', '_', regex=True)
hitters_data["Last Name"] = hitters_data["Last Name"].replace('-', '_', regex=True)
pitchers_data["Last Name"] = pitchers_data["Last Name"].replace('-', '_', regex=True)


In [10]:
#data preprocessing
c_set = hitters_data[hitters_data["Position"]=="C"]
oneb_set = hitters_data[hitters_data["Position"].str.contains("1B")]
twob_set = hitters_data[hitters_data["Position"].str.contains("2B")]
threeb_set = hitters_data[hitters_data["Position"].str.contains("3B")]
ss_set = hitters_data[hitters_data["Position"].str.contains("SS")]
of_set = hitters_data[hitters_data["Position"].str.contains("OF")]

c_list = c_set["Last Name"].unique()
oneb_list = oneb_set["Last Name"].unique()
twob_list = twob_set["Last Name"].unique()
threeb_list = threeb_set["Last Name"].unique()
ss_list = ss_set["Last Name"].unique()
of_list = of_set["Last Name"].unique()
p_list = pitchers_data["Last Name"].unique()

#hitter
hitters_list = hitters_data["Last Name"].unique()
all_player_list = list(set(list(p_list) + list(hitters_list)))
#team
team_list = hitters_data["Team"].unique()
#line


#we do not want hitters who is not in order 1,23,4
#non_first_powerline_list = (skater_data[ (skater_data["Power_Play"]!="P1") & (skater_data["Position"]=="D") ]
#                            ["Last Name"].unique())
#def mu: each player has a projection score
mu={}
for index,row in hitters_data.iterrows():
    mu[row["Last Name"]] = row.Projection
for index,row in pitchers_data.iterrows():
    mu[row["Last Name"]] = row.Projection
#def salary
salary_dict = {}
for index,row in hitters_data.iterrows():
    salary_dict[row["Last Name"]] = row.Salary
for index,row in pitchers_data.iterrows():
    salary_dict[row["Last Name"]] = row.Salary
#def team_dict
team_dict = {}
for team in team_list:
    team_dict[team] = []
for index, row in hitters_data.iterrows():
    team_dict[row.Team].append(row["Last Name"])
    team_dict[row.Team] = list(set( team_dict[row.Team]))
for index, row in pitchers_data.iterrows():
    team_dict[row.Team].append(row["Last Name"])
    team_dict[row.Team] = list(set( team_dict[row.Team]))
#def pitchers's opponent set
p_opponent_dict = {}
for x in p_list:
    p_opponent_dict[x]=[]
for index, row in pitchers_data.iterrows():
    p_opponent_dict[row["Last Name"]] = row["Opponent"]
# def team dict {team1:[player1..]}
hitters_team_dict = {}
for x in team_list:
    hitters_team_dict[x] = []
for index,row in hitters_data.iterrows():
    hitters_team_dict[row.Team].append(row["Last Name"])
    hitters_team_dict[row.Team] = list(set(hitters_team_dict[row.Team] ))

batting_order_list = ["1","2","3","4","5","6","7","8","9"]
hitters_data = hitters_data[hitters_data["Batting Order (Confirmed)"].isin(batting_order_list)]

order_stack=[ [1,2,3,4,5],
              [2,3,4,5,6],
              [3,4,5,6,7],
              [5,6,7,8,9],
              [6,7,8,9,1],
              [7,8,9,1,2],
              [8,9,1,2,3],
              [9,1,2,3,4]]
order_stack_dict={}
for team in team_list:
    for i in range(8):
        order_stack_dict[team,i] = []

for index,row in hitters_data.iterrows():
    for i in range(8):
        if int(row["Batting Order (Confirmed)"]) in order_stack[i]:
            order_stack_dict[row.Team,i].append(row["Last Name"])

In [11]:
hitters_data.shape

(135, 31)

## output automation
#### parse output to dk's template format

In [12]:
template_id_df = pd.read_csv("DKSalaries (1).csv")
template_id_df["Unnamed: 13"] = template_id_df["Unnamed: 13"].str.replace(' ', '_', regex=True)
template_id_df["Unnamed: 13"] = template_id_df["Unnamed: 13"].str.replace('-', '_', regex=True)
player_id_dict = {}
for index,row in template_id_df.iterrows():
    player_id_dict[row["Unnamed: 13"]] = row["Unnamed: 14"]
player_score_dict = {}
for index,row in hitters_data.iterrows():
    player_score_dict[row["Last Name"]] = row["Actual FP"]
for index,row in pitchers_data.iterrows():
    player_score_dict[row["Last Name"]] = row["Actual FP"]

In [56]:
K=800
%time out_put = sequential_opt(K,7, optimization_form_4, 4) # glpk

1-th entry started>>>
2-th entry started>>>
3-th entry started>>>
4-th entry started>>>
5-th entry started>>>
6-th entry started>>>
7-th entry started>>>
8-th entry started>>>
9-th entry started>>>
10-th entry started>>>
11-th entry started>>>
12-th entry started>>>
13-th entry started>>>
14-th entry started>>>
15-th entry started>>>
16-th entry started>>>
17-th entry started>>>
18-th entry started>>>
19-th entry started>>>
20-th entry started>>>
21-th entry started>>>
22-th entry started>>>
23-th entry started>>>
24-th entry started>>>
25-th entry started>>>
26-th entry started>>>
27-th entry started>>>
28-th entry started>>>
29-th entry started>>>
30-th entry started>>>
31-th entry started>>>
32-th entry started>>>
33-th entry started>>>
34-th entry started>>>
35-th entry started>>>
36-th entry started>>>
37-th entry started>>>
38-th entry started>>>
39-th entry started>>>
40-th entry started>>>
41-th entry started>>>
42-th entry started>>>
43-th entry started>>>
44-th entry started>

347-th entry started>>>
348-th entry started>>>
349-th entry started>>>
350-th entry started>>>
351-th entry started>>>
352-th entry started>>>
353-th entry started>>>
354-th entry started>>>
355-th entry started>>>
356-th entry started>>>
357-th entry started>>>
358-th entry started>>>
359-th entry started>>>
360-th entry started>>>
361-th entry started>>>
362-th entry started>>>
363-th entry started>>>
364-th entry started>>>
365-th entry started>>>
366-th entry started>>>
367-th entry started>>>
368-th entry started>>>
369-th entry started>>>
370-th entry started>>>
371-th entry started>>>
372-th entry started>>>
373-th entry started>>>
374-th entry started>>>
375-th entry started>>>
376-th entry started>>>
377-th entry started>>>
378-th entry started>>>
379-th entry started>>>
380-th entry started>>>
381-th entry started>>>
382-th entry started>>>
383-th entry started>>>
384-th entry started>>>
385-th entry started>>>
386-th entry started>>>
387-th entry started>>>
388-th entry sta

689-th entry started>>>
690-th entry started>>>
691-th entry started>>>
692-th entry started>>>
693-th entry started>>>
694-th entry started>>>
695-th entry started>>>
696-th entry started>>>
697-th entry started>>>
698-th entry started>>>
699-th entry started>>>
700-th entry started>>>
701-th entry started>>>
702-th entry started>>>
703-th entry started>>>
704-th entry started>>>
705-th entry started>>>
706-th entry started>>>
707-th entry started>>>
708-th entry started>>>
709-th entry started>>>
710-th entry started>>>
711-th entry started>>>
712-th entry started>>>
713-th entry started>>>
714-th entry started>>>
715-th entry started>>>
716-th entry started>>>
717-th entry started>>>
718-th entry started>>>
719-th entry started>>>
720-th entry started>>>
721-th entry started>>>
722-th entry started>>>
723-th entry started>>>
724-th entry started>>>
725-th entry started>>>
726-th entry started>>>
727-th entry started>>>
728-th entry started>>>
729-th entry started>>>
730-th entry sta

In [44]:

DK_df = []
for i in range(K-1):
    out_c=[]
    out_p=[]
    out_oneb=[]
    out_twob=[]
    out_threeb=[]
    out_ss=[]
    out_of=[]

    row = out_put.iloc[i]
    for player in row:
        if player in c_list:
            out_c.append(player) 
        elif player in p_list:
            out_p.append(player)
        elif player in oneb_list:
            out_oneb.append(player)
        elif player in twob_list:
            out_twob.append(player)
        elif player in threeb_list:
            out_threeb.append(player)
        elif player in ss_list:
            out_ss.append(player)
        elif player in of_list:
            out_of.append(player)
    dk_out_row = out_p + out_c + out_oneb + out_twob + out_threeb + out_ss + out_of
    DK_df.append(dk_out_row)
    
DK_df = pd.DataFrame(DK_df)     


In [45]:
DK_df.replace(player_id_dict,inplace=True)
DK_df.columns = ["P","P","C","1B","2B","3B","SS","OF","OF","OF"]
DK_df.to_csv("dk_result 3_31.csv",index = False)

In [46]:
ex_list = []
for index,row in out_put.iterrows():
    score = 0
    for x in row:
        try:
            score+=player_score_dict[x]
        except:
            pass
        
    if score >=130:
        ex_list.append(score)
        print(index)
        print(score)
pd.DataFrame(ex_list).to_csv("ex_50_setting2.csv")

entry8
149.39999999999998
entry9
135.6
entry11
146.6
entry14
140.4
entry17
140.8
entry20
158.6
entry32
151.4
entry34
156.6
entry37
134.6
entry39
135.4
entry42
136.8
entry46
138.6
entry53
141.6
entry71
169.6
entry76
136.8
entry79
139.6
entry80
134.4
entry89
140.8
entry97
138.6
entry100
142.4
entry112
138.0
entry118
131.4
entry120
135.4
entry123
142.39999999999998
entry127
149.6
entry128
147.4
entry136
145.4
entry137
130.6
entry149
153.4
entry152
132.39999999999998
entry155
136.4
entry166
134.39999999999998
entry171
139.4
entry202
143.2
entry225
145.6
entry233
136.4
entry239
151.4
entry256
145.4
entry265
156.2
entry270
142.6
entry272
133.4
entry274
138.6
entry285
146.4


In [47]:
print("Max: " + str(max(ex_list)))
print("Mean: " + str(sum(ex_list)/len(ex_list)))

Max: 169.6
Mean: 142.41860465116272


In [48]:
line_vis = out_put.iloc[40]
print(line_vis)


0    Justin_Verlander
1     Kendrys_Morales
2       Mitch_Haniger
3          Dee_Gordon
4           Jay_Bruce
5     Domingo_Santana
6    Martin_Maldonado
7        Matt_Chapman
8      Dansby_Swanson
9       Julio_Teheran
Name: entry41, dtype: object


In [49]:
pitchers_data[pitchers_data["Last Name"].isin(line_vis)]

,Last Name,Likes,Position,Salary,Team,Opponent,Vegas Odds Win,Vegas Odds RA,Vegas Odds K,Opponent wOBA,...,Wind Factor,L5 FP,L80 FP,S FP,Floor FP,Ceil FP,Projection,Proj Val,Actual FP,Actual Val
0,Justin_Verlander,9.0,SP,10400,HOU,NYY,57%,3.64,8.0,0.328,...,0%,17.6,17.6,17.6,-3.0,38.2,23.3,2.2,14.1,1.4
14,Julio_Teheran,0.0,SP,5800,ATL,COL,49%,5.29,6.0,0.311,...,0%,16.8,16.8,16.8,15.2,18.3,14.1,2.4,4.5,0.8


In [50]:
hitters_data[hitters_data["Last Name"].isin(line_vis)]

,Last Name,Likes,Position,Salary,Team,Opponent,RS,Pitcher Name,Pitcher wOBA,Pitcher wOBA Diff,...,Wind Factor,L10 FP,L80 FP,S FP,Floor FP,Ceil FP,Projection,Proj Val,Actual FP,Actual Val
15,Matt_Chapman,7.0,3B,4800,OAK,BAL,5.58,Cashner,0.332,2%,...,0%,6.8,6.8,6.8,1.4,12.2,9.4,2.0,9.0,1.9
20,Mitch_Haniger,7.0,OF,4600,SEA,KC,5.04,Bailey,0.376,11%,...,0%,9.7,9.7,9.7,4.3,15.1,10.1,2.2,7.0,1.5
40,Domingo_Santana,6.0,OF,4300,SEA,KC,5.04,Bailey,0.376,11%,...,0%,13.4,13.4,13.4,3.9,23.0,9.2,2.1,21.0,4.9
45,Dansby_Swanson,5.0,SS,4300,ATL,COL,5.21,Freeland,0.324,0%,...,0%,10.4,10.4,10.4,3.6,17.3,8.9,2.1,16.0,3.7
53,Kendrys_Morales,6.0,1B,4300,OAK,BAL,5.58,Cashner,0.350,6%,...,0%,4.0,4.0,4.0,1.1,6.9,8.6,2.0,3.0,0.7
64,Jay_Bruce,6.0,OF,4200,SEA,KC,5.04,Bailey,0.380,13%,...,0%,12.4,12.4,12.4,0.4,24.4,9.3,2.2,14.0,3.3
81,Dee_Gordon,3.0,2B,4100,SEA,KC,5.04,Bailey,0.380,13%,...,0%,7.9,7.9,7.9,3.0,12.7,8.3,2.0,10.0,2.4
283,Martin_Maldonado,1.0,C,3200,KC,SEA,4.96,Hernandez,0.330,2%,...,0%,3.7,3.7,3.7,0.3,7.2,5.5,1.7,0.0,0.0


In [51]:
hitters_data[hitters_data["Last Name"].isin(line_vis)][["Team",'Batting Order (Confirmed)']]

,Team,Batting Order (Confirmed)
15,OAK,2
20,SEA,1
40,SEA,2
45,ATL,6
53,OAK,5
64,SEA,3
81,SEA,9
283,KC,8


In [52]:
sum(out_put.iloc[9].replace(player_score_dict))

114.6

In [53]:
#137 8:56

In [54]:
###
# for people with two postion assume both postition could fit, exclude 1 at end
# for hitter <= 3 team
# for hitter, at least one team has >=4/5 player
# hitter consecutive order

In [55]:
out_put

,0,1,2,3,4,5,6,7,8,9
entry1,Justin_Verlander,Adam_Eaton,Josh_Donaldson,Edwin_Encarnacion,Dee_Gordon,Omar_Narvaez,Jay_Bruce,Domingo_Santana,Dansby_Swanson,Julio_Teheran
entry2,Adam_Eaton,Mitch_Haniger,Josh_Donaldson,Freddie_Freeman,Omar_Narvaez,Anibal_Sanchez,Ozzie_Albies,Ronald_Acuna_Jr.,Julio_Teheran,Corey_Seager
entry3,Miles_Mikolas,Mitch_Haniger,Josh_Donaldson,Adalberto_Mondesi,Edwin_Encarnacion,Omar_Narvaez,Jay_Bruce,Domingo_Santana,Ozzie_Albies,Julio_Teheran
entry4,Jesus_Sucre,Josh_Donaldson,Adalberto_Mondesi,Freddie_Freeman,Jay_Bruce,Domingo_Santana,Anibal_Sanchez,Ozzie_Albies,Ronald_Acuna_Jr.,Julio_Teheran
entry5,Joc_Pederson,Jesus_Sucre,Justin_Verlander,Mitch_Haniger,Josh_Donaldson,Edwin_Encarnacion,Dee_Gordon,Jay_Bruce,Julio_Teheran,Corey_Seager
entry6,Jesus_Sucre,Justin_Verlander,Mitch_Haniger,Scott_Kingery,Edwin_Encarnacion,Jay_Bruce,Domingo_Santana,Dansby_Swanson,Ozzie_Albies,Julio_Teheran
entry7,Miles_Mikolas,Rhys_Hoskins,Bryce_Harper,Adam_Eaton,Josh_Donaldson,Jean_Segura,Omar_Narvaez,Ozzie_Albies,Andrew_McCutchen,Julio_Teheran
entry8,Miles_Mikolas,Mitch_Haniger,Josh_Donaldson,Edwin_Encarnacion,Dee_Gordon,Omar_Narvaez,Jay_Bruce,Anibal_Sanchez,Dansby_Swanson,Ronald_Acuna_Jr.
entry9,Mitch_Haniger,Josh_Donaldson,Eric_Lauer,Edwin_Encarnacion,Jay_Bruce,Domingo_Santana,Anibal_Sanchez,Chris_Iannetta,Dansby_Swanson,Ozzie_Albies
entry10,Justin_Verlander,Hunter_Renfroe,Mitch_Haniger,Josh_Donaldson,Edwin_Encarnacion,Jean_Segura,Dee_Gordon,Domingo_Santana,Francisco_Mejia,Julio_Teheran
